In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [ ]:
import io
train = pd.read_csv('train.csv')
train.head()
test = pd.read_csv('test.csv')

In [ ]:
train["condition"].replace(np.nan, -1, inplace = True)
test["condition"].replace(np.nan,-1, inplace = True)

In [ ]:
c=['color_type','condition']
for i in c:
    loc = np.append(train[i].values, test[i].values, axis=0)
    from sklearn.preprocessing import LabelEncoder
    l=LabelEncoder()
    l.fit(list(set(loc)))
    train[i]=l.transform(train[i])
    test[i]=l.transform(test[i])

In [ ]:
train['issue_date'] = pd.to_datetime(train['issue_date'])
train['listing_date'] = pd.to_datetime(train['listing_date'])
train['days'] = (train['listing_date'] - train['issue_date']).dt.days 
train['number_months'] = ((train.listing_date - train.issue_date)/np.timedelta64(1, 'M')).astype('int')
train['number_year'] = ((train.listing_date - train.issue_date)/np.timedelta64(1, 'Y')).astype('int')
train = train.drop(['pet_id','issue_date','listing_date'],axis=1)
train 

,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category,days,number_months,number_year
0,3,18,0.80,7.78,13,9,0.0,1,73,2,0
1,2,53,0.72,14.19,13,9,0.0,2,1862,61,5
2,0,15,0.15,40.90,15,4,2.0,4,752,24,2
3,2,53,0.62,17.82,0,1,0.0,2,755,24,2
4,3,2,0.50,11.06,18,4,0.0,1,52,1,0
...,...,...,...,...,...,...,...,...,...,...,...
18829,3,52,0.44,27.36,0,1,0.0,2,407,13,1
18830,0,15,0.73,14.25,15,4,2.0,4,386,12,1
18831,1,22,0.99,28.13,13,9,1.0,1,2954,97,8
18832,1,48,0.55,44.82,13,9,1.0,2,272,8,0


In [ ]:
test['issue_date'] = pd.to_datetime(test['issue_date'])
test['listing_date'] = pd.to_datetime(test['listing_date'])
test['days'] = (test['listing_date'] - test['issue_date']).dt.days 
test['number_months'] = ((test.listing_date - test.issue_date)/np.timedelta64(1, 'M')).astype('int')
test['number_year'] = ((test.listing_date - test.issue_date)/np.timedelta64(1, 'Y')).astype('int')

test

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,days,number_months,number_year
0,ANSL_75005,2005-08-17,2017-09-07 15:35:00,1,2,0.87,42.73,0,7,4404,144,12
1,ANSL_76663,2018-11-15,2019-05-08 17:24:00,2,38,0.06,6.71,0,1,174,5,0
2,ANSL_58259,2012-10-11,2018-04-02 16:51:00,2,2,0.24,41.21,0,7,1999,65,5
3,ANSL_67171,2015-02-13,2018-04-06 07:25:00,2,2,0.29,8.46,7,1,1148,37,3
4,ANSL_72871,2017-01-18,2018-04-26 13:42:00,2,15,0.71,30.92,0,7,463,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...
8067,ANSL_66809,2016-02-10,2017-03-10 14:56:00,3,15,0.82,36.08,13,9,394,12,1
8068,ANSL_59041,2015-12-07,2018-02-12 00:00:00,1,48,0.49,27.54,13,9,798,26,2
8069,ANSL_60034,2015-12-08,2017-01-04 17:19:00,1,2,0.98,37.19,0,7,393,12,1
8070,ANSL_58066,2016-06-28,2017-07-20 18:19:00,0,2,0.79,23.83,0,2,387,12,1


In [ ]:
test.drop(['issue_date','listing_date'], axis = 1, inplace = True)

In [ ]:
train['height(m)'] = train['height(cm)'].apply(lambda x: x/100 )
test['height(m)'] = test['height(cm)'].apply(lambda x: x/100 )

In [ ]:
del train['height(cm)']
del test['height(cm)']

In [ ]:
train['area'] = train['length(m)'] * train['height(m)']
test['area'] = test['length(m)'] * test['height(m)']

In [ ]:
train['average'] = train[['X1', 'X2']].mean(axis=1)
test['average'] = test[['X1', 'X2']].mean(axis=1)

In [ ]:
train['X_product'] = train['X1'] * train['X2']
test['X_product'] = test['X1'] * test['X2']

In [ ]:
Test = test.copy()

In [ ]:
Test.drop('pet_id',inplace = True, axis = 1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, learning_curve, ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier, IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix

In [ ]:
y1 = train['breed_category']
y2 = train['pet_category']
X = train.drop(columns=[ 'breed_category','pet_category'])
X1 = X[["condition","color_type","days","X2","X1"]]
X2 = X[["condition","color_type","days","X2","X1","area"]]
X3 = X[["condition","color_type","days","X2","X1","average","number_months","X_product"]]
X4 = X[["days","color_type","height(m)","length(m)","condition","area"]]

In [ ]:
Test1= Test[["condition","color_type","days","X2","X1"]]
Test2 = Test[["condition","color_type","days","X2","X1","area"]]
Test3 = Test[["condition","color_type","days","X2","X1","average","number_months","X_product"]]
Test4 = Test[["days","color_type","height(m)","length(m)","condition","area"]]

In [ ]:
lgbmc2 = LGBMClassifier(random_state=44, n_estimators=2000, colsample_bytree=0.9, max_depth=31,  objective='multiclass',
                       learning_rate=0.01,reg_alpha=0, reg_lambda=1, boosting_type='gbdt')
cross_val_score(lgbmc2, X1, y1, cv=5, n_jobs=-1, verbose=1, scoring='f1_weighted').mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   36.9s finished


0.9073955527297354

In [ ]:
import matplotlib.pyplot as plt


lgbmc2.fit(X,y2)
print(lgbmc2.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(lgbmc2.feature_importances_, index=X.columns)
feat_importances.nlargest(10)

[14138 52720 70839 14312 16065 71925]


area          71925
days          70839
color_type    52720
X1            16065
X2            14312
condition     14138
dtype: int64

In [ ]:
lgbmc = LGBMClassifier(random_state=44, n_estimators=2000, colsample_bytree=0.9, max_depth=31,  objective='multiclass',
                       learning_rate=0.01,reg_alpha=0, reg_lambda=1, boosting_type='gbdt',cv = 40,n_jobs=-1, verbose=1, scoring='f1_weighted')
cross_val_score(lgbmc, X1, y2, cv=40, n_jobs=-1, verbose=1, scoring='f1_weighted').mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  8.0min finished


0.9048292576226027

In [ ]:
y_pred = cross_val_predict(lgbmc, X1, y2, cv=40, n_jobs=-1, verbose=1).mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


In [ ]:
y_pred 

In [ ]:
lgbmc2.fit(X1,y1)
pred1 = lgbmc2.predict(Test1)

In [ ]:
lgbmc.fit(X1,y2)
pred2 = lgbmc.predict(Test1)

In [ ]:
submission = pd.DataFrame({
        "pet_id": test["pet_id"],
        "breed_category": pred1,
        "pet_category": pred2
    })

In [ ]:
submission.to_csv("3.csv",index = False)

In [ ]:
submission2.to_csv("O.csv",index = False)

In [ ]:
lgbmc2 = LGBMClassifier(boosting_type='gbdt',
        colsample_bytree=0.9234, learning_rate=0.1, max_depth=31,
        metric='None', min_child_samples=399, min_child_weight=0.1,
        min_split_gain=0.0, n_estimators=5000, n_jobs=4, num_leaves=13,
        objective='multiclass', random_state=314, reg_alpha=2, reg_lambda=5,
        subsample=0.855, subsample_for_bin=200000,
        subsample_freq=0)
cross_val_score(lgbmc2, X1, y2, cv=15, n_jobs=-1, verbose=1, scoring='f1_weighted').mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  7.8min finished


0.9020418053653849